# Creating and Testing Tools for the FPL Gaffer

## FPL API Data Retrieval Tool

In [1]:
import httpx

In [2]:
base_url = "https://fantasy.premierleague.com/api"
client = httpx.Client()

### Retrieving Bootstrap Data

In [3]:
# Retrieve bootstrap data from FPL API endpoint
bootstrap_endpoint = f"{base_url}/bootstrap-static/"
response = client.get(bootstrap_endpoint)

# Ensure the request was successful
response.raise_for_status()
response_data = response.json()

# Display the keys of the response data
response_data.keys()

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements'])

### Getting Player Data

In [4]:
player_data = response_data["elements"]

# Display the first 10 players
for player in player_data[:10]:
    print(f"ID: {player["id"]}, Name: {player["web_name"]}, Team: {player["team"]}, Position: {player["element_type"]}")

ID: 1, Name: Raya, Team: 1, Position: 1
ID: 2, Name: Arrizabalaga, Team: 1, Position: 1
ID: 3, Name: Hein, Team: 1, Position: 1
ID: 4, Name: Setford, Team: 1, Position: 1
ID: 5, Name: Gabriel, Team: 1, Position: 2
ID: 6, Name: Saliba, Team: 1, Position: 2
ID: 7, Name: Calafiori, Team: 1, Position: 2
ID: 8, Name: J.Timber, Team: 1, Position: 2
ID: 9, Name: Kiwior, Team: 1, Position: 2
ID: 10, Name: Lewis-Skelly, Team: 1, Position: 2


### Getting Gameweek Data

In [7]:
gw_data = response_data.get("events", [])

# Since no gameweek is active now as the season hasn't started,
# let's get the other data than current gameweek

gw_dict = {}
for gw in gw_data:
    # Get the details for the firt gameweek
    gw_dict["gamweek"] = gw.get("id") if gw else None
    gw_dict["deadline"] = gw.get("deadline_time") if gw else None
    gw_dict["finished"] = gw.get("finished") if gw else None
    break

gw_dict

{'gamweek': 1, 'deadline': '2025-08-15T17:30:00Z', 'finished': False}

### Getting Fixture Data